# Black Scholes Model

The Black-Scholes model was published in 1973 for pricing options on non-dividend-paying stocks. Since then, it has revolutionized quantitative finance and laid the foundation for modern derivatives pricing. The Black-Scholes model is based on a number of assumptions about how financial markets operate, including:

Arbitrage-free markets

Frictionless and continuous trading

A constant risk-free interest rate

Log-normally distributed asset price movements

Constant volatility

While these assumptions may not hold true in reality, they are not necessarily restrictive. The generalized Black-Scholes framework has been extended to price derivatives on other asset classes, such as the Black-76 model for commodity futures and the Garman-Kohlhagen model for foreign exchange (FX) options. These models remain widely used in derivative pricing and risk management today.

**Black Scholes Formula**

The Black–Scholes equation describes the price of the option over time as

$$
\frac{\partial V}{\partial t}
+ \frac 1{2}{\sigma^2 S^2} \frac{\partial^2 V}{\partial S^2}
+ r S \frac{\partial V}{\partial S} - rV = 0 
$$

<br>Solving the above equation, we know that the value of a call option for a non-dividend paying stock is:<br>

$$ C = SN(d_1) - Ke^{-rt}N(d_2) $$

and, the corresponding put option price is:

$$ P = Ke^{-rt}N(-d_2) - SN(-d_1)$$

where, 

$$ d_1= \frac{1}{\sigma \sqrt{t}}\left[\ln{\left(\frac{S}{K}\right)} +{\left(r + \frac{\sigma^2}{2}\right)}t\right] $$
<br>

$$ d_2= d_1 - \sigma \sqrt{t} $$
<br>

$$ N(x)=\frac{1}{\sqrt{2\pi}} \int_{-\infty}^{x} \mathrm e^{-\frac{1}{2}x^2} dx $$


$S$ is the spot price of the underlying asset<br>
$K$ is the strike price<br>
$r$ is the annualized continuous compounded risk free rate<br>
$\sigma$ is the volatility of returns of the underlying asset<br>
$t$ is time to maturity (expressed in years)<br>
$N(x)$ is the standard normal cumulative distribution<br>


**Greeks**

| **Description** |  | **Greeks for Call Option** | **Greeks for Put Option**             
|:-----------|:----------------|:----------------|:--------                                                                                                       
| **Delta**  |$ \space\space \frac {\partial V}{\partial S}$ Sensitivity of Value to changes in price | $N(d_1)$ | $-N(-d_1)$
| **Gamma**  |$ \space\space \frac {\partial ^{2}V}{\partial S^{2}}$ Sensitivity of Delta to changes in price | $\frac{N'(d_1)}{S\sigma\sqrt{t}}$
| **Vega**   |$ \space\space \frac {\partial V}{\partial \sigma}$ Sensitivity of Value to changes in volatility | $SN'(d_1)\sqrt{t}$
| **Theta**  |$ \space\space \frac {\partial V}{\partial t}$ Sensitivity of Value to changes in time | $-\frac{SN'(d_1)\sigma}{2\sqrt{t}}-rKe^{-rt}N(d_2)$ | $-\frac{SN'(d_1)\sigma}{2\sqrt{t}}+rKe^{-rt}N(-d_2)$
| **Rho**    |$ \space\space \frac {\partial V}{\partial r}$  Sensitivity of Value to changes in risk-free | $Kte^{-rt}N(d_2)$ | $-Kte^{-rt}N(-d_2)$

**Import Libraries**

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from scipy.stats import norm
from tabulate import tabulate

**Calculate Option Price**

In [ ]:
# Define function for pricing
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    '''Black Scholes Option Pricing'''
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

In [ ]:
# BS Price 
black_scholes_price(100,100,1,0.05,0.2,option_type='put')

**Calculate Greeks**

In [ ]:
def compute_greeks(S, K, T, r, sigma, option_type='call'):
    '''Compute option greeks'''
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    delta = norm.cdf(d1) if option_type == 'call' else -norm.cdf(-d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100  
    
    theta_call = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)) / 365
    theta_put = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)) / 365
    
    rho_call = K * T * np.exp(-r * T) * norm.cdf(d2) / 100
    rho_put = -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100

    theta = theta_call if option_type == 'call' else theta_put
    rho = rho_call if option_type == 'call' else rho_put

    return {'Delta': delta, 'Gamma': gamma, 'Vega': vega, 'Theta': theta, 'Rho': rho}


In [ ]:
compute_greeks(100,100,1,0.05,0.2,option_type="put")

## Quantmod Option Module

The quantmod option module offers a versatile and comprehensive toolkit for valuing a wide range of financial derivatives. It includes multiple pricing models implemented in Python such as Black-Scholes, Binomial, and Monte Carlo methods, enabling accurate computation of theoretical prices for European, American, and exotic options, including Asian and barrier options.

Built with a clean, object-oriented architecture and leveraging Pydantic for smart function calling and input validation, the module allows users to easily specify input parameters, configure model-specific settings like time steps or simulation paths, handle early exercise features, and generate precise valuations all through a unified and intuitive Python API.

Its flexible design makes it ideal for creating intelligent option agents and building custom MCP servers by harnessing the power of function calling.

Key Features:

* OptionInputs: A structured class to standardize and validate input parameters.

* BlackScholesOptionPricing: Implements the Black-Scholes model for pricing options and calculating Greeks.

* BinomialOptionPricing: Prices options using the binomial tree method, including support for early exercise.

* MonteCarloOptionPricing: Uses Monte Carlo simulation to price European, Asian, and barrier options.

Let's now price the option Black Scholes Options using quantmod 

In [ ]:
# Import Quantmod Option Module
from quantmod.models import OptionInputs, BlackScholesOptionPricing

In [ ]:
# Define option inputs parameters
inputs=OptionInputs(
    spot=100,
    strike=100,
    ttm=1,
    rate=0.05,
    volatility=0.2
    )

# Initialize the BS Option Pricing Engine
option = BlackScholesOptionPricing(inputs=inputs)

In [ ]:
# Print the BS option price
header = ['Option Price', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho']
table = [
    [option.call_price, option.call_delta, option.gamma, option.call_theta, option.vega, option.call_rho],
    [option.put_price, option.put_delta, option.gamma, option.put_theta, option.vega, option.put_rho]
]

print(tabulate(table,header))

## Visualise Payoff

In [ ]:
# Import opstrat
import opstrat as op

**Single Payoff: Call Option**

In [ ]:
# Plot option payoff
op.single_plotter(
    op_type='c',
    spot=inputs.spot,
    spot_range=25,
    strike=inputs.strike,
    tr_type='b',
    op_pr=option.call_price
)

**Multi-leg Payoff: Straddle**

In [ ]:
# Straddle
op_1 = {'op_type': 'c', 'strike':inputs.strike, 'tr_type': 's', 'op_pr': option.call_price}
op_2 = {'op_type': 'p', 'strike':inputs.strike, 'tr_type': 's', 'op_pr': option.put_price}

# plot
op_list = [op_1, op_2]
op.multi_plotter(spot=inputs.spot, spot_range=25, op_list=op_list)

**Multi-leg Payoff: Strangle**

In [ ]:
# Strangle
leg1_inputs =  OptionInputs(spot=100,strike=110,ttm=1,rate=0.05,volatility=0.2) 
leg1 = BlackScholesOptionPricing(leg1_inputs)

leg2_inputs =  OptionInputs(spot=100,strike=90,ttm=1,rate=0.05,volatility=0.2) 
leg2 = BlackScholesOptionPricing(leg2_inputs)
    
op_1 = {'op_type': 'c', 'strike':leg1_inputs.strike, 'tr_type': 's', 'op_pr': leg1.call_price}
op_2 = {'op_type': 'p', 'strike':leg2_inputs.strike, 'tr_type': 's', 'op_pr': leg2.put_price}

# plot
op_list = [op_1, op_2]
op.multi_plotter(spot=inputs.spot, spot_range=25, op_list=op_list)

**Multi-leg Payoff: Ironfly**

In [ ]:
# Ironfly
leg2_inputs =  OptionInputs(spot=100,strike=90,ttm=1,rate=0.05,volatility=0.2) 
leg2 = BlackScholesOptionPricing(leg2_inputs)

leg3_inputs =  OptionInputs(spot=100,strike=110,ttm=1,rate=0.05,volatility=0.2) 
leg3 = BlackScholesOptionPricing(leg3_inputs)

op_1 = {'op_type': 'c', 'strike':inputs.strike, 'tr_type': 's', 'op_pr': option.call_price}
op_2 = {'op_type': 'p', 'strike':inputs.strike, 'tr_type': 's', 'op_pr': option.put_price}
op_3 = {'op_type': 'p', 'strike':leg2_inputs.strike, 'tr_type': 'b', 'op_pr': leg2.put_price}
op_4 = {'op_type': 'c', 'strike':leg3_inputs.strike, 'tr_type': 'b', 'op_pr': leg3.call_price}

op_list = [op_1, op_2, op_3, op_4]
op.multi_plotter(spot=inputs.spot, spot_range=25, op_list=op_list)

## Greeks Analysis

We'll now retrieve option chain for SPY for August 2025 expiration from yahoo finance and manipulate the option chain to perform our analysis. 

https://finance.yahoo.com/quote/SPY250829C00620000/

In [ ]:
# Import quantmod
from quantmod.markets import getTicker
import quantmod.charts
from datetime import datetime

In [ ]:
# Get SPY option chain
spy = getTicker('SPY')
options = spy.option_chain('2025-08-29')
dte = (datetime(2025, 8, 29) - datetime.today()).days/365

In [ ]:
# August 2025 620 SPY call option price
spy_opt = BlackScholesOptionPricing(OptionInputs(spot=620,strike=620,ttm=dte,rate=0.0,volatility=0.2107))

print(f'Option Price of SPY250829C00620000 with BS Model is {spy_opt.call_price:0.4f}')

In [ ]:
# Filter calls for strike at or above 600 and puts at or below 650 
df = options.calls[(options.calls['strike'] >= 600) & (options.calls['strike'] <= 650)].copy()
df = df[['strike', 'lastPrice']]
# df.reset_index(drop=True, inplace=True)

# Add implied volatility and initialize Greeks
df['IV'] = 0.20
df['Delta'] = 0.0
df['Gamma'] = 0.0
df['Vega'] = 0.0
df['Theta'] = 0.0

for idx, row in df.iterrows():
    opt = BlackScholesOptionPricing(
        OptionInputs(
            spot=620,
            strike=row['strike'],
            ttm=dte,
            rate=0.0,
            volatility=row['IV']
        )
    )

    df.at[idx, 'Delta'] = opt.call_delta
    df.at[idx, 'Gamma'] = opt.gamma
    df.at[idx, 'Vega'] = opt.vega
    df.at[idx, 'Theta'] = opt.call_theta
    
# Set index
df.set_index('strike', inplace=True)
df.head(2)

**Visualise Greeks Vs Strike Behaviour**

In [ ]:
# Subplots of greeks
df[['Delta', 'Gamma', 'Vega', 'Theta']].iplot(kind="subplots", title="Greeks Vs Strikes", showlegend=True)

---
[Kannan Singaravelu](https://www.linkedin.com/in/kannansi) | Refer [Quantmod](https://kannansingaravelu.com/quantmod/) and [Opstrat](https://github.com/hashabcd/opstrat) for more information.